In [1]:
import numpy as np
import dask.dataframe as dd

In [20]:
path="dataset/train_balanced.csv"
df=dd.read_csv(path)
df = df.drop(['Unnamed: 0'],axis=1)
df.head()

,label,uid,task_id,adv_id,creat_type_cd,adv_prim_id,dev_id,inter_type_cd,slot_id,spread_app_id,...,list_time,device_price,up_life_duration,up_membership_grade,membership_life_duration,consume_purchase,communication_onlinerate,communication_avgonline_30d,indu_name,pt_d
0,0,2078912,4265,2140,7,167,37,5,11,44,...,8,5,20,1,-1,2,0^1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18...,13,17,6
1,0,1029564,4305,4663,7,156,56,5,21,58,...,10,2,16,-1,-1,2,0^1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18...,13,17,5
2,0,1952180,2229,5483,7,156,56,5,13,58,...,5,4,-1,-1,-1,2,5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^...,11,17,2
3,0,2046795,3506,4867,2,142,36,5,12,80,...,14,2,12,-1,-1,2,6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,7,42,6
4,1,2013717,5879,3563,7,150,17,5,16,11,...,8,5,-1,-1,-1,2,5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^...,11,36,2


## dummies

In [22]:
df = df.categorize()
df.head()

,label,uid,task_id,adv_id,creat_type_cd,adv_prim_id,dev_id,inter_type_cd,slot_id,spread_app_id,...,list_time,device_price,up_life_duration,up_membership_grade,membership_life_duration,consume_purchase,communication_onlinerate,communication_avgonline_30d,indu_name,pt_d
0,0,2078912,4265,2140,7,167,37,5,11,44,...,8,5,20,1,-1,2,0^1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18...,13,17,6
1,0,1029564,4305,4663,7,156,56,5,21,58,...,10,2,16,-1,-1,2,0^1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18...,13,17,5
2,0,1952180,2229,5483,7,156,56,5,13,58,...,5,4,-1,-1,-1,2,5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^...,11,17,2
3,0,2046795,3506,4867,2,142,36,5,12,80,...,14,2,12,-1,-1,2,6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,7,42,6
4,1,2013717,5879,3563,7,150,17,5,16,11,...,8,5,-1,-1,-1,2,5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^...,11,36,2


In [26]:
#categorical_variables = df[['gender', 'city', 'career', 'residence', 'device_name', 'task_id', 'adv_prim_id', 'dev_id', 'spread_app_id', 'tags','indu_name','adv_id','inter_type_cd', 'creat_type_cd']]
df = dd.get_dummies(df, columns=['gender'])
df = dd.get_dummies(categorical_variables)
df.head()

AttributeError: 'Series' object has no attribute 'categorize'

In [16]:
df['communication_onlinerate']=df['communication_onlinerate'].apply(lambda x:x.split('^'))
dummy=dd.get_dummies(df['communication_onlinerate'].apply(dd.Series).stack()).sum(level=0)
df = dd.concat([df, dummy], axis=1)
df = df.drop('communication_onlinerate',axis=1)
df.head()

KeyError: 'communication_onlinerate'

## Bins

In [ ]:
df['device_size_binned']=dd.cut(df['device_size'], 5,labels=False)
df['emui_dev_binned']=dd.cut(df['emui_dev'], 5,labels=False)
df['list_time_binned']=dd.cut(df['list_time'], 5,labels=False)

In [ ]:
x = df.loc[:, df.columns != 'label']
y = df.loc[:, df.columns == 'label']
y=np.array(y)

## Standard

In [10]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [11]:
scaler=MinMaxScaler()
scaler.fit(x)
x=scaler.transform(x)

MemoryError: 

## Classifier

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
k = 5
kf = KFold(n_splits=k)

'''
models = {"Logistic Regression":LogisticRegression(multi_class="auto", solver="lbfgs", max_iter=2000), \
          "Decision Tree": DecisionTreeClassifier(), \
          "Random Forest": RandomForestClassifier(n_estimators=100), \
          "Nearest Neighbors": KNeighborsClassifier(n_neighbors=5), \
          "Perceptron": MLPClassifier(max_iter=2000)}
'''

models = {"Logistic Regression":LogisticRegression(multi_class="auto", solver="lbfgs", max_iter=2000), \
          "Decision Tree": DecisionTreeClassifier(), \
          "Random Forest": RandomForestClassifier(n_estimators=100), \
          "Nearest Neighbors": KNeighborsClassifier(n_neighbors=5), \
          "Perceptron": MLPClassifier(max_iter=2000)}

for name in models:
    print("Start training models of {}".format(name))
    i = 0
    f1_global = []
    for train, test in kf.split(x):
        i+=1
        x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]
        model = models[name].fit(x_train,y_train)
        y_out = model.predict(x_test)
        print("Log Loss",metrics.log_loss(y_test,LR.predict_proba(X_test)))
        # We have to provide the real value of the target and the model's output
        f1_global.append(f1_score(y_test, y_out, average="weighted"))
        print(confusion_matrix(y_test, y_out))
        print(classification_report(y_test, y_out))
    print("Global: f1={:.4f}\n".format(np.average(f1_global)))